In [1]:
import os, sys, re, json, time, shutil
import itertools
import collections
from IPython.display import display

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf

# Pandas because pandas are awesome, and for pretty-printing
import pandas as pd
# Set pandas floating point display
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# Helper libraries for this notebook
import utils; reload(utils)
import vocabulary; reload(vocabulary)
import rnnlm; reload(rnnlm)

<module 'rnnlm' from 'rnnlm.pyc'>

In [2]:
import tensorflow as tf
import rnnlm; reload(rnnlm)

# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)


with tf.Graph().as_default():
  tf.set_random_seed(42)

  lm = rnnlm.RNNLM(V=10000, H=200, num_layers=2)
  lm.BuildCoreGraph()
  lm.BuildTrainGraph()
  lm.BuildSamplerGraph()

  summary_writer = tf.train.SummaryWriter("tf_summaries", 
                                          tf.get_default_graph())
dev_sents = {}

In [3]:
def run_epoch(lm, session, batch_iterator, train=False,
              verbose=False, tick_s=10, 
              keep_prob=1.0, learning_rate=0.1):
  start_time = time.time()
  tick_time = start_time  # for showing status
  total_cost = 0.0  # total cost, summed over all words
  total_words = 0

  if train:
    train_op = lm.train_step_
    keep_prob = keep_prob
    loss = lm.train_loss_
  else:
    train_op = tf.no_op()
    keep_prob = 1.0  # no dropout at test time
    loss = lm.loss_  # true loss, if train_loss is an approximation

  for i, (w, y) in enumerate(batch_iterator):
    cost = 0.0
    #### YOUR CODE HERE ####
    np.random.seed(42)

    # At first batch in epoch, get a clean intitial state
    if i == 0:
        h = session.run(lm.initial_h_, {lm.input_w_: w})
 
    cost, h, _ = session.run([loss, lm.final_h_, train_op], feed_dict= {lm.target_y_: y, lm.initial_h_:h,
        lm.input_w_: w, lm.dropout_keep_prob_:keep_prob, lm.learning_rate_:learning_rate})      
    
    #### END(YOUR CODE) ####
    total_cost += cost
    total_words += w.size  # w.size = batch_size * max_time

    ##
    # Print average loss-so-far for epoch
    # If using train_loss_, this may be an underestimate.
    if verbose and (time.time() - tick_time >= tick_s):
      avg_cost = total_cost / total_words
      avg_wps = total_words / (time.time() - start_time)
      print "[batch %d]: seen %d words at %d wps, loss = %.3f" % (i,
          total_words, avg_wps, avg_cost)
      tick_time = time.time()  # reset time ticker

  return total_cost / total_words

In [19]:
# Load the dataset
import time
os.environ['TZ'] = 'US/Pacific'
print time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())
lm_name = 'neg'


V = 10000
vocab, train_ids, test_ids, dev_sents_by_cat = utils.load_corpus('movie_reviews', split1=0.8, split2=0.10, V=V, categories=lm_name, shuffle=42)
dev_sents[lm_name] = dev_sents_by_cat
print time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())

Sat, 17 Dec 2016 15:16:58
Loaded 35495 sentences (751256 tokens)
Training set: 28396 sentences (601237 tokens)
Test set: 3549 sentences (74607 tokens)
Dev set: 3550 sentences (75412 tokens)
Sat, 17 Dec 2016 15:17:08


In [20]:
# Training parameters
max_time = 20
batch_size = 50
learning_rate = 0.5
keep_prob = 1.0
num_epochs = 10

# Model parameters
model_params = dict(V=V, 
                    H=100, 
                    num_layers=1)

trained_filename = './tf_saved/rnnlm_trained' + '_' + lm_name

In [21]:
def score_dataset(lm, session, ids, name="Data"):
  bi = utils.batch_generator(ids, batch_size=100, max_time=100)
  cost = run_epoch(lm, session, bi, 
                   learning_rate=1.0, keep_prob=1.0, 
                   train=False, verbose=False, tick_s=3600)
  print "%s: avg. loss: %.03f  (perplexity: %.02f)" % (name, cost, np.exp(cost))

In [22]:
# Will print status every this many seconds
print_interval = 5
print time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())
# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)

with tf.Graph().as_default(), tf.Session() as session:
  # Seed RNG for repeatability
  tf.set_random_seed(42)
  
  with tf.variable_scope("model", reuse=None):
    lm = rnnlm.RNNLM(**model_params)
    lm.BuildCoreGraph()
    lm.BuildTrainGraph()
  
  session.run(tf.initialize_all_variables())
  saver = tf.train.Saver()
  
  for epoch in xrange(1,num_epochs+1):
    t0_epoch = time.time()
    bi = utils.batch_generator(train_ids, batch_size, max_time)
    print "[epoch %d] Starting epoch %d" % (epoch, epoch)
    #### YOUR CODE HERE ####

    run_epoch(lm, session, bi, train=True, keep_prob=keep_prob, learning_rate=learning_rate)

    #### END(YOUR CODE) ####
    print "[epoch %d] Completed in %s" % (epoch, utils.pretty_timedelta(since=t0_epoch))
    
    ##
    # score_dataset will run a forward pass over the entire dataset
    # and report perplexity scores. This can be slow (around 1/2 to 
    # 1/4 as long as a full epoch), so you may want to comment it out
    # to speed up training on a slow machine. Be sure to run it at the 
    # end to evaluate your score.
    print ("[epoch %d]" % epoch),
    score_dataset(lm, session, train_ids, name="Train set")
    print ("[epoch %d]" % epoch),
    score_dataset(lm, session, test_ids, name="Test set")
    print ""
    
    # Save a checkpoint
    saver.save(session, './tf_saved/rnnlm' + '_' + lm_name, global_step=epoch)
    print time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())
  # Save final model
  saver.save(session, trained_filename)

Sat, 17 Dec 2016 15:17:10
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


[epoch 1] Starting epoch 1
[epoch 1] Completed in 0:01:34
[epoch 1] Train set: avg. loss: 5.252  (perplexity: 191.04)
[epoch 1] Test set: avg. loss: 5.401  (perplexity: 221.62)

Sat, 17 Dec 2016 15:21:43
[epoch 2] Starting epoch 2
[epoch 2] Completed in 0:01:33
[epoch 2] Train set: avg. loss: 4.971  (perplexity: 144.18)
[epoch 2] Test set: avg. loss: 5.230  (perplexity: 186.83)

Sat, 17 Dec 2016 15:26:05
[epoch 3] Starting epoch 3
[epoch 3] Completed in 0:01:31
[epoch 3] Train set: avg. loss: 4.803  (perplexity: 121.88)
[epoch 3] Test set: avg. loss: 5.152  (perplexity: 172.70)

Sat, 17 Dec 2016 15:30:09
[epoch 4] Starting epoch 4
[epoch 4] Completed in 0:01:24
[epoch 4] Train set: avg. loss: 4.678  (perplexity: 107.56)
[epoch 4] Test set: avg. loss: 5.106  (perplexity: 165.03)

Sat, 17 Dec 2016 15:34:01
[epoch 5] Starting epoch 5
[epoch 5] Completed in 0:01:24
[epoch 5] Train set: avg. loss: 4.590  (perplexity: 98.51)
[epoch 5] Test set: avg. loss: 5.085  (perplexity: 161.61)

Sat, 17

In [24]:
def sample_step(lm, session, input_w, initial_h):
  """Run a single RNN step and return sampled predictions.
  
  Args:
    lm : rnnlm.RNNLM
    session: tf.Session
    input_w : [batch_size] list of indices
    initial_h : [batch_size, hidden_dims]
  
  Returns:
    final_h : final hidden state, compatible with initial_h
    samples : [batch_size, 1] vector of indices
  """
  #### YOUR CODE HERE ####
  # Reshape input to column vector
  input_w = np.array(input_w, dtype=np.int32).reshape([-1,1])
  
  # Run sample ops
  final_h, samples = session.run([lm.final_h_, lm.pred_samples_], 
        feed_dict={lm.input_w_: input_w, lm.initial_h_: initial_h, lm.dropout_keep_prob_: 1.0, lm.learning_rate_:0.1})
  
  #### END(YOUR CODE) ####
  return final_h, samples[:,-1,:]

In [25]:
# Same as above, but as a batch
reload(rnnlm)
max_steps = 20
num_samples = 10
random_seed = 42

with tf.Graph().as_default(), tf.Session() as session:
  # Seed RNG for repeatability
  tf.set_random_seed(random_seed)

  with tf.variable_scope("model", reuse=None):
    lm = rnnlm.RNNLM(**model_params)
    lm.BuildCoreGraph()
    lm.BuildSamplerGraph()

  # Load the trained model
  saver = tf.train.Saver()
  saver.restore(session, trained_filename)

  # Make initial state for a batch with batch_size = num_samples
  w = np.repeat([[vocab.START_ID]], num_samples, axis=0)
  h = session.run(lm.initial_h_, {lm.input_w_: w})
  # We'll take one step for each sequence on each iteration 
  for i in xrange(max_steps):
    h, y = sample_step(lm, session, w[:,-1:], h)
    w = np.hstack((w,y))

  # Print generated sentences
  for row in w:
    for i, word_id in enumerate(row):
      print vocab.id_to_word[word_id],
      if (i != 0) and (word_id == vocab.START_ID):
        break
    print ""

<s> amanda has gone wife in the where stereotypes include an oscar day , <unk> lost in the film , bruce 
<s> jessica ? <s> 
<s> played by you know whether it having made any other fails . <s> 
<s> long ever ' s the novel are a comedy , but cable after and female lead by slow to deal 
<s> escapes an amoral satisfying conclusion and ron <unk> ( in seattle , jesse ( the camera spent the males and 
<s> first movies that ' s a " double team of the tv - <unk> , well <unk> brody and plays 
<s> or getting nothing not of yes , september <unk> ) get close without it is seen this movies , and 
<s> the end result , i ' television star or not getting my pick - or apocalyptic movie that ' s 
<s> many other reason that <unk> . <s> 
<s> my town and performs running into the feel left any idea is everywhere lines , more plots ) , " 


In [26]:
def score_seq(lm, session, seq, vocab):
  """Score a sequence of words. Returns total log-probability."""
  padded_ids = vocab.words_to_ids(utils.canonicalize_words(["<s>"] + seq, 
                                                           wordset=vocab.word_to_id))
  w = np.reshape(padded_ids[:-1], [1,-1])
  y = np.reshape(padded_ids[1:],  [1,-1])
  h = session.run(lm.initial_h_, {lm.input_w_: w})
  feed_dict = {lm.input_w_:w,
               lm.target_y_:y,
               lm.initial_h_:h,
               lm.dropout_keep_prob_: 1.0}
  # Return log(P(seq)) = -1*loss
  return -1*session.run(lm.loss_, feed_dict)

def load_and_score(inputs, sort=False, silent=False):
  """Load the trained model and score the given words."""
  with tf.Graph().as_default(), tf.Session() as session:  
    with tf.variable_scope("model", reuse=None):
      lm = rnnlm.RNNLM(**model_params)
      lm.BuildCoreGraph()
        
    # Load the trained model
    saver = tf.train.Saver()
    saver.restore(session, trained_filename)
  
    if isinstance(inputs[0], str) or isinstance(inputs[0], unicode):
      inputs = [inputs]

    # Actually run scoring
    results = []
    for words in inputs:
      score = score_seq(lm, session, words, vocab)
      results.append((score, words))
    
    # Sort if requested
    if sort: results = sorted(results, reverse=True)
    
    # Print results
    for score, words in results:
      if not silent:
          print "\"%s\" : %.05f" % (" ".join(words), score)
    if silent:
      return [score for score, words in results]

In [27]:
sents = ["once upon a time",
         "the quick brown fox jumps over the lazy dog"]
load_and_score([s.split() for s in sents])

"once upon a time" : -20.58923
"the quick brown fox jumps over the lazy dog" : -77.79940


In [28]:
#### YOUR CODE HERE ####
sents = ["the boy and the girl are",
         "the boy and the girl is",
         
         "the boys are", 
         "the boys is",
         
         "there are many deer", 
         "there are many deers"]
load_and_score([s.split() for s in sents])

#### END(YOUR CODE) ####

"the boy and the girl are" : -29.20731
"the boy and the girl is" : -27.89507
"the boys are" : -13.95618
"the boys is" : -12.79805
"there are many deer" : -12.77805
"there are many deers" : -12.77805


In [29]:
#### YOUR CODE HERE ####

sents = ["peanuts are my favorite kind of nut",
         "peanuts are my favorite kind of vegetable",
         
         "when I'm hungry I really prefer to eat",
         "when I'm hungry I really prefer to drink",
         
         "as nut",
         "as vegetable",
         
         "like nut", 
         "like vegetable",
         
         "peanut as nut",
         "peanut as vegetable"
         ]
load_and_score([s.split() for s in sents])


#### END(YOUR CODE) ####

"peanuts are my favorite kind of nut" : -31.22075
"peanuts are my favorite kind of vegetable" : -31.22075
"when I'm hungry I really prefer to eat" : -61.23142
"when I'm hungry I really prefer to drink" : -62.50514
"as nut" : -6.62594
"as vegetable" : -6.62594
"like nut" : -8.61763
"like vegetable" : -8.61763
"peanut as nut" : -10.71208
"peanut as vegetable" : -10.71208


In [30]:
prefix = "I have lots of".split()
noun = "toys"
adjectives = ["square", "green", "plastic"]
inputs = []
for adjs in itertools.permutations(adjectives):
  words = prefix + list(adjs) + [noun]
  inputs.append(words)
    
load_and_score(inputs, sort=True)

"I have lots of square plastic green toys" : -60.67012
"I have lots of square green plastic toys" : -62.83763
"I have lots of plastic square green toys" : -65.46872
"I have lots of green square plastic toys" : -67.34250
"I have lots of plastic green square toys" : -67.68961
"I have lots of green plastic square toys" : -72.32302


In [31]:
prefix = "I have lots of".split()
noun = "toys"
adjectives = ["flat", "green", "plastic"]
inputs = []
for adjs in itertools.permutations(adjectives):
  words = prefix + list(adjs) + [noun]
  inputs.append(words)
    
load_and_score(inputs, sort=True)

"I have lots of flat green plastic toys" : -61.12477
"I have lots of plastic flat green toys" : -61.94820
"I have lots of flat plastic green toys" : -63.83598
"I have lots of plastic green flat toys" : -68.84360
"I have lots of green flat plastic toys" : -70.38685
"I have lots of green plastic flat toys" : -73.16902


In [32]:
prefix = "I have lots of".split()
noun = "toys"
adjectives = ["round", "green", "plastic"]
inputs = []
for adjs in itertools.permutations(adjectives):
  words = prefix + list(adjs) + [noun]
  inputs.append(words)
    
load_and_score(inputs, sort=True)

"I have lots of plastic round green toys" : -59.85853
"I have lots of plastic green round toys" : -64.61606
"I have lots of green plastic round toys" : -65.59953
"I have lots of round green plastic toys" : -65.97829
"I have lots of round plastic green toys" : -66.72260
"I have lots of green round plastic toys" : -69.62610


In [33]:
prefix = "I have lots of".split()
noun = "toys"
adjectives = ["small", "green", "plastic"]
inputs = []
for adjs in itertools.permutations(adjectives):
  words = prefix + list(adjs) + [noun]
  inputs.append(words)
print inputs
load_and_score(inputs, sort=True)

[['I', 'have', 'lots', 'of', 'small', 'green', 'plastic', 'toys'], ['I', 'have', 'lots', 'of', 'small', 'plastic', 'green', 'toys'], ['I', 'have', 'lots', 'of', 'green', 'small', 'plastic', 'toys'], ['I', 'have', 'lots', 'of', 'green', 'plastic', 'small', 'toys'], ['I', 'have', 'lots', 'of', 'plastic', 'small', 'green', 'toys'], ['I', 'have', 'lots', 'of', 'plastic', 'green', 'small', 'toys']]
"I have lots of small green plastic toys" : -58.50835
"I have lots of small plastic green toys" : -58.67347
"I have lots of plastic green small toys" : -61.02421
"I have lots of plastic small green toys" : -61.09280
"I have lots of green plastic small toys" : -63.32803
"I have lots of green small plastic toys" : -65.33598


In [34]:
print time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())
#neg predictor
total_pos = 0
total_neg = 0
tn = 0
fn = 0
tp = 0
fp = 0
#num_sample = 10000
nopt = 0
nont = 0

lm_name = 'pos'
trained_filename = './tf_saved/rnnlm_trained' + '_' + lm_name

pos_for_pos_score = load_and_score(dev_sents['pos'], silent=True)
neg_for_pos_score = load_and_score(dev_sents['neg'], silent=True)

print time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())


lm_name = 'neg'
trained_filename = './tf_saved/rnnlm_trained' + '_' + lm_name

pos_for_neg_score = load_and_score(dev_sents['pos'], silent=True)
neg_for_neg_score = load_and_score(dev_sents['neg'], silent=True)

Sat, 17 Dec 2016 16:02:50
Sat, 17 Dec 2016 16:03:59


In [35]:
for i in range(len(dev_sents['pos'])):
    
    if pos_for_pos_score[i] > pos_for_neg_score[i]:
        tp = tp + 1
    else:
        if pos_for_pos_score[i] < pos_for_neg_score[i]:
            fp = fp + 1
        else: #flip a coin
            nopt = nopt + 1
            if np.random.randint(0, 1) == 0:
                tp = tp + 1
            else: 
                fp = fp + 1

for i in range(len(dev_sents['neg'])):

    if neg_for_pos_score[i] < neg_for_neg_score[i]:
        tn = tn + 1
    else:
        if neg_for_pos_score[i] > neg_for_neg_score[i]:
            fn = fn + 1
        else: #flip a coin
            nont = nont + 1
            if np.random.randint(0, 1) == 0:
                tn = tn + 1
            else:
                fn = fn + 1

print "tp = %d, fp = %d, tn = %d, fn = %d, nopt = %d, nont = %d\n" % (tp, fp, tn, fn, nopt, nont)

precision = tp * 1.0 / (tp+fp)
recall = tp * 1.0 / (tp+fn)
accuracy = (tp + tn) * 1.0 / (tp+fp+tn+fn)
true_neg_rate = tn * 1.0 / (tn + fp)
f_measure = 2.0 * precision * recall / (precision + recall)

print "precision = %.2f, recall = %.2f, accuracy = %.2f, true_neg_rate = %.2f, f-measure = %.2f\n" % \
      (precision*100, recall*100, accuracy*100, true_neg_rate*100, f_measure*100)

tp = 2519, fp = 1085, tn = 2054, fn = 1496, nopt = 0, nont = 0

precision = 69.89, recall = 62.74, accuracy = 63.92, true_neg_rate = 65.43, f-measure = 66.12



In [36]:
print len(dev_sents['pos']), len(dev_sents['neg']), len(pos_for_pos_score), len(pos_for_neg_score), len(neg_for_pos_score), len(neg_for_neg_score)

3604 3550 3604 3604 3550 3550


In [55]:
import numpy as np
np.random.seed(3)

mr = nltk.corpus.movie_reviews
pos = [i for i in mr.sents(categories='pos')]
neg = [i for i in mr.sents(categories='neg')]

rng = np.random.RandomState(True)
rng.shuffle(pos)  # in-place
rng.shuffle(neg)

print time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())
#neg predictor
total_pos = 0
total_neg = 0
tn = 0
fn = 0
tp = 0
fp = 0
num_sample = 10000

lm_name = 'pos'
trained_filename = './tf_saved/rnnlm_trained' + '_' + lm_name

pos_for_pos_score = load_and_score(pos[:num_sample], silent=True)
neg_for_pos_score = load_and_score(neg[:num_sample], silent=True)

lm_name = 'neg'
trained_filename = './tf_saved/rnnlm_trained' + '_' + lm_name

pos_for_neg_score = load_and_score(pos[:num_sample], silent=True)
neg_for_neg_score = load_and_score(neg[:num_sample], silent=True)


for i in range(num_sample):
    
    if pos_for_pos_score[i] > pos_for_neg_score[i]:
        tp = tp + 1
    else:
        if pos_for_pos_score[i] < pos_for_neg_score[i]:
            fp = fp + 1
        else: #flip a coin
            nopt = nopt + 1
            if np.random.randint(0, 1) == 0:
                tp = tp + 1
            else: 
                fp = fp + 1

    if neg_for_pos_score[i] < neg_for_neg_score[i]:
        tn = tn + 1
    else:
        if neg_for_pos_score[i] > neg_for_neg_score[i]:
            fn = fn + 1
        else: #flip a coin
            if np.random.randint(0, 1) == 0:
                tn = tn + 1
            else:
                fn = fn + 1

print tp, fp, tn, fn, num_sample, num_sample
print time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime())

Sat, 17 Dec 2016 12:56:52
8386 1614 8544 1456 10000 10000
Sat, 17 Dec 2016 13:03:30


In [56]:

precision = tp * 1.0 / (tp+fp)
recall = tp * 1.0 / (tp+fn)
accuracy = (tp + tn) * 1.0 / (tp+fp+tn+fn)
true_neg_rate = tn * 1.0 / (tn + fp)
f_measure = 2.0 * precision * recall / (precision + recall)

print "precision = %.2f, recall = %.2f, accuracy = %.2f, true_neg_rate = %.2f, f-measure = %.2f\n" % \
      (precision*100, recall*100, accuracy*100, true_neg_rate*100, f_measure*100)
    

precision = 83.86, recall = 85.21, accuracy = 84.65, true_neg_rate = 84.11, f-measure = 84.53

